In [ ]:
## Upload files to OpenAI

from openai import OpenAI
client = OpenAI()
import time
import os
import glob

stop = True
for filename in [os.path.abspath(f) for f in glob.iglob('./docx_docs/**/*', recursive=True)]:
    if ".docx" in filename:
        print(filename)
        time.sleep(1)
        file = client.files.create(
            file=open(filename, "rb"),
            purpose='assistants'
        )


In [ ]:
# Fetch file names

from openai import OpenAI
import json
client = OpenAI()

!rm files.json
!curl https://api.openai.com/v1/files -H "Authorization: Bearer $OPENAI_API_KEY" >> files.json
json_file = json.load(open('files.json'))

print(json_file['data'])

In [99]:
# Assistant fetching helper function

from helper import all_assistants

my_assistants = all_assistants()

In [ ]:
# Remove all assistants from OpenAI

for asst in my_assistants:
    response = client.beta.assistants.delete(asst.id)
    print(response)
    time.sleep(0.1)

In [ ]:
# Assistant creation

asst_file = []
for file in json_file['data']:
    if file['filename'].split('.')[-1] == 'docx':
        id = file['id'].split('-')[-1]
        print(file['id'])
        assistant = client.beta.assistants.create(
            name=f"assistant-{id}",
            instructions="""You are a chatbot that answers questions about a document written by Nikola Tesla. You can summarize the document or answer questions about it, if you do not know the answer reply "I'm sorry, I don't know the answer.", don't try to make up something.""",
            model="gpt-3.5-turbo-1106",
            tools=[{"type": "retrieval"}],
            file_ids=[file['id']]
        )
        print({'assistant_id': assistant.id, 'file_id': file['id']})
        asst_file.append({'assistant_id': assistant.id, 'file_id': file['id']})
        # Save asst id and file id in dict
        time.sleep(0.1)

In [80]:
import json
with open('asst_file.json', 'w') as f:
    json.dump(asst_file, f)

In [54]:
from openai import OpenAI
client = OpenAI()

a = client.files.list()

In [58]:
docx_files = [file for file in a.data if file.filename.split(".")[-1] == "docx"]

In [60]:
len(docx_files)

359

In [ ]:
# Metadata ssistant creation

from openai import OpenAI
client = OpenAI()

file = client.files.create(
    file=open('metadata.json', "rb"),
    purpose='assistants'
)

assistant = client.beta.assistants.create(
    name=f"assistant-metadata",
    instructions="""You are a chatbot that answers questions about a json document that contains data about lectures, patents and articles writen by Nikola Tesla. 'type' field of the data entry tells what type of document it is, 'date' when it was written, 'source' is the source in case of articles and 'register_num' is the patent regustration number. If you do not have the data to answer answer reply "I'm sorry, I don't know the answer.", don't try to make up something.""",
    model="gpt-3.5-turbo-1106",
    tools=[{"type": "retrieval"}],
    file_ids=[file.id]
)